# Nba game predictions and betting 

The following notebook helps making perdiction on the outcome (Win vs Loss) of a basketball game.
The data gathered here are retrieved from my own webscrapping.
Some of the packages used are beautiful soup / regex etc (for more insight and upon request, codes can be shared) 
The major websites used are wikipedia, https://www.sports-reference.com/cbb/, https://www.basketball-reference.com/, ESPN.

While webscrapping, data engineering, determinitic models and previous predictions based on the knowledge of the game make a major part of this final product, such prepocessing will not be discussed but could be upon request.

Some data: 

- The age, physicality and experience of the team are aggregated into the feature: H_W_aggr_ ,  Age_Yr_aggr_

- Ratings from the previous game are used for defense, offense, shooting etc. and a weight is given for a team on a given day

- Exhaustion and Success are computed via score diff, successive victories, game played, game difference with last game.

- When teams face each other, each features for team 1 are joined to features for team 2 and the outcome win or loss is predicted with probability

- For games happening in the future, deterministic models devised by myself and other such as rolling mean etc help in the determing or predicting the features used for the predictive models (Win/Loss) here.


Data from 2000 to 2019 are used here. The results, methods and models here were used on the pre-covid 2020 (events happening in the future at the time) season with a close to 60% success rate.


In [1]:
#import xgboost
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from datetime import datetime, timedelta

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import fbeta_score, make_scorer
from sklearn import svm


Seasons1 = pd.read_excel('Seasons.xlsx', encoding = 'latin-1')
Seasons2 = pd.read_excel('Seasons2.xlsx', encoding = 'latin-1')
Seasons3 = pd.read_excel('Seasons3.xlsx', encoding = 'latin-1')
Seasons4 = pd.read_excel('Seasons4.xlsx', encoding = 'latin-1')

Seasons_all = pd.concat([Seasons1, Seasons2, Seasons3])
#Seasons_all = pd.concat([Seasons1, Seasons2, Seasons3, Seasons4])
Seasons_all.reset_index(inplace = True)
len(Seasons_all)

43918

In [2]:
a_ind = Seasons_all[['Date_x', 'Win_x','Loss_x']].drop_duplicates().index
Seasons_all = Seasons_all[Seasons_all.index.isin(a_ind)]
len(Seasons_all)

21959

# Features

The feautures created are shown below

In [3]:
Seasons_all.columns.values

array(['index', 'Unnamed: 0', 'Unnamed: 0.1', 'Date_x', 'Win_x', 'Loss_x',
       'Up_x', 'Down_x', 'Age_Yr_aggr_Win', 'H_W_aggr_Win', 'TS%_Win',
       'OFFRTG_Win', 'DEFRTG_Win', 'Age_Yr_aggr_Loss', 'H_W_aggr_Loss',
       'TS%_Loss', 'OFFRTG_Loss', 'DEFRTG_Loss', 'Game_Number',
       'Successive_Vic', 'Days_Diff', 'Score_diff', 'Vic',
       'Game_Number_2', 'Successive_Vic_2', 'Days_Diff_2', 'Score_diff_2',
       'Vic_2', 'Age_Yr_aggr_1', 'H_W_aggr_1', 'TS%_1', 'OFFRTG_1',
       'DEFRTG_1', 'Age_Yr_aggr_2', 'H_W_aggr_2', 'TS%_2', 'OFFRTG_2',
       'DEFRTG_2', 'Results_1'], dtype=object)

In [4]:
Seasons_all.iloc[:,18:].columns.values

array(['Game_Number', 'Successive_Vic', 'Days_Diff', 'Score_diff', 'Vic',
       'Game_Number_2', 'Successive_Vic_2', 'Days_Diff_2', 'Score_diff_2',
       'Vic_2', 'Age_Yr_aggr_1', 'H_W_aggr_1', 'TS%_1', 'OFFRTG_1',
       'DEFRTG_1', 'Age_Yr_aggr_2', 'H_W_aggr_2', 'TS%_2', 'OFFRTG_2',
       'DEFRTG_2', 'Results_1'], dtype=object)

# Features and rows

After the features are selected the raws are selected since prior analysis suggests game changes on average every 5 years. 

We then operate different regressions and gridSearch to fin the probabilty of win/loss.


In [5]:
Mod_seasons = Seasons_all.iloc[:,18:].loc[35000:]
len(Mod_seasons)

#Mod_seasons = Seasons_all.iloc[:,18:]

4130

In [6]:
X = Mod_seasons.loc[:,Mod_seasons.columns != 'Results_1'] # Trainning columns
y = Mod_seasons.Results_1 #testing columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) # split for trainning

scaler = MinMaxScaler()


X_train_Scl1 = scaler.fit_transform(X_train) #scaling the data since the distances and the weights 
X_test_Scl1 = scaler.transform(X_test)


#X_train_Scl1 = X_train 
#X_test_Scl1 = X_test

#clf1 = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1)
#clf1.fit(X_train,y_train)

lr = LogisticRegression(penalty='l2', max_iter=40, tol=5)
#lr.fit(X_train,y_train)

grid = { 'C': np.power(10.0, np.arange(-30, 30)), 'solver': ['lbfgs'] }

grid_AUC_scores = GridSearchCV(lr, param_grid = grid, scoring = 'roc_auc')
grid_AUC_scores.fit(X_train_Scl1, y_train)

Accuracy = GridSearchCV(lr, grid, scoring='accuracy')
Accuracy.fit(X_train_Scl1, y_train)

#optimize the scores ROC: sensitivity and AUC: should be above .5 and closer to 1
#to differentiate from a dummy classifier. In order to optimize our classification
print('AUC:', grid_AUC_scores.best_score_, ', accuracy:', Accuracy.best_score_ , ', params:',grid_AUC_scores.best_params_)

#, ', accuracy:', Accuracy.best_score_




AUC: 0.7499321196812267 , accuracy: 0.6909917140028141 , params: {'C': 1000.0, 'solver': 'lbfgs'}


Forest trees can help refine the model by doing feature selection (Not much gain)

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

clf1 = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)

print("All Columns Accuracy:")

# View The Accuracy Of Our Full Feature (4 Features) Model
print(accuracy_score(y_test, y_pred))

print("All Columns AUC:")
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
print(metrics.auc(fpr, tpr))

sfm1 = SelectFromModel(clf1, threshold=0.06)
sfm1.fit(X_train, y_train)

#We set a threshold value and find the important variables
#give important variables for noise reduction
print("important Columns:")

for feature_list_index in sfm1.get_support(indices=True):
    print(X_train.columns[feature_list_index])

X_important_train = sfm1.transform(X_train)
X_important_test = sfm1.transform(X_test)

clf1.fit(X_important_train,y_train)
y_important_pred1 = clf1.predict(X_important_test)

print("important Columns Accuracy:")
# View The Accuracy Of Our Limited Feature (2 Features) Model
print(accuracy_score(y_test, y_important_pred1))

print("important Columns AUC:")
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_important_pred1)
metrics.auc(fpr, tpr)

All Columns Accuracy:
0.6631171345595354
All Columns AUC:
0.6644191378208743
important Columns:
TS%_1
OFFRTG_1
DEFRTG_1
TS%_2
OFFRTG_2
DEFRTG_2
important Columns Accuracy:
0.665053242981607
important Columns AUC:


0.6656631713458865

In [8]:
from sklearn import svm



Cs = [0.1, 1, 10] #C values for simplicity of model
gammas = [0.01, 1, 10] #We will trade off the influence far or close of the model with C
nfolds = 5
grid2 = {'C': Cs, 'gamma' : gammas}
#grid_search = GridSearchCV(svm.SVC(kernel='rbf'), grid2, cv=nfolds)
#grid_search.fit(X_train_Scl1, y_train)
grid_search_AUC = GridSearchCV(svm.SVC(kernel='rbf'), grid2, cv=nfolds, scoring='roc_auc')
grid_search_AUC.fit(X_train_Scl1, y_train)
print(grid_search_AUC.best_params_,', AUC:', grid_search_AUC.best_score_) # best parameters


{'C': 10, 'gamma': 0.01} , AUC: 0.748990286924857


In [9]:
from sklearn.metrics import accuracy_score
# we do a linear regression on the filtered set
#The model behaves badly. The relationship is very likely nonlinear and a gridsearch will likely not improve the 
#scores so much
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
linreg = linear_model.Lasso(alpha=.5)
linreg.fit(X_train_Scl1, y_train)
PredLinY = linreg.predict(X_test_Scl1) 
print('AUC:', sklearn.metrics.roc_auc_score(y_test, PredLinY.round()), ', accuracy:', accuracy_score(y_test, PredLinY.round()))

AUC: 0.5 , accuracy: 0.4743465634075508


In [10]:
from sklearn import svm
#scaler = MinMaxScaler()
X = Mod_seasons.loc[:,Mod_seasons.columns != 'Results_1'] # Trainning columns
y = Mod_seasons.Results_1 #testing columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) # split for trainning

scaler = MinMaxScaler()


X_train_Scl1 = scaler.fit_transform(X_train) #scaling the data since the distances and the weights 
X_test_Scl1 = scaler.transform(X_test)



#X_scl = scaler.fit_transform(X)
#Y = X_scl
model = svm.SVC(kernel='rbf', gamma = 1, C = 1, probability=True)
model.fit(X_train_Scl1, y_train)
#Y = model.predict(X_scl)

SVC(C=1, gamma=1, probability=True)

In [11]:
#2018-2019 check

Seasons4.reset_index(inplace = True)
a_ind_1 = Seasons4[['Date_x', 'Win_x','Loss_x']].drop_duplicates().index
Seasons4 = Seasons4[Seasons4.index.isin(a_ind_1)]
Mod_seasons_check = Seasons4.iloc[:,18:]
X_ch = Mod_seasons_check.loc[:,Mod_seasons_check.columns != 'Results_1']
y_ch = Mod_seasons_check.Results_1
X_ch_scl = scaler.fit_transform(X_ch)
lr = LogisticRegression(penalty='l2', max_iter=40, tol=5, C = 1, solver = 'lbfgs' )
lr.fit(X_train_Scl1, y_train)
#model = svm.SVC(kernel='rbf', gamma = 1, C = 1, probability=True)

LogisticRegression(C=1, max_iter=40, tol=5)

# Monetary Gains of individual

In the first column below, we use logistic regression to make predictions on the 2019 games with the command:

Y_ch = lr.predict(X_ch_scl)



In the second cell below we have the command:
  
     for i in range(len(Proba[:35])):

with len(Proba[:35]) helping us find a range of game on which to be placing bets (35 first games of 2019) we find out that placing bets of $20 on each game would help us achieve total gains of $95 (or a 34% interest rate) 
compared to placing bets on the entire season of 2019 that yield $1036 at the end of the season, or an interest rate of approximately 8.8%

Note that better performance can be achieved with SVC




In [12]:
Y_ch = lr.predict(X_ch_scl)
Y_ch = pd.Series(Y_ch)
y_ch1 = y_ch.reset_index()

diff = []

for i in range(len(Y_ch)):
    d = Y_ch[i]-y_ch1.Results_1.loc[i]
    diff.append(d)

diff = abs(pd.Series(diff))
diff.sum()/len(diff)

0.47784045124899277

In [13]:
Y_ch_1 = []
ind_w = []
ind_l = []
gain = []
loss = []
Proba = lr.predict_proba(X_ch_scl)
for i in range(len(Proba[:35])):
    if Proba[i][1] > .5:
        Y_ch_1.append(1)
        if Proba[i][1] > .7:
            ind_w.extend([[i,1,Proba[i][1]]])
    else:
        Y_ch_1.append(0)
        if Proba[i][0] > .7:
            ind_l.extend([[i,0,Proba[i][0]]])
            
diff = []

for i in range(len(Y_ch_1)):
    d = Y_ch_1[i]-y_ch1.Results_1.loc[i]
    #if d != 0
    diff.append(d)

diff = abs(pd.Series(diff))
diff.sum()/len(diff)

for i in range(len(ind_w)):
    if y_ch1.Results_1.loc[ind_w[i][0]] - 1 == 0:
        gain.append(ind_w[i][2])
    else: 
        loss.append(-1*ind_w[i][2])
for i in range(len(ind_l)):
    if y_ch1.Results_1.loc[ind_l[i][0]] - 0 == 0:
        gain.append(ind_l[i][2])
    else: 
        loss.append(-1*ind_l[i][2])   


In [14]:
print((pd.Series(loss).sum()+pd.Series(gain).sum())*20, len(loss), len(gain), 20*(len(loss)+len(gain)))
#print('The interest rate is:', (pd.Series(loss[:50]).sum()+pd.Series(gain[:50]).sum())*100/len(loss[:50]),'%')
print('The interest rate is:', (pd.Series(loss).sum()+pd.Series(gain).sum())*100/(len(loss)+len(gain)),'%')


95.41398875811194 4 10 280
The interest rate is: 34.076424556468545 %


Note: the games quotes are not considered here.
